In [408]:
import pandas as pd
import numpy as np

In [409]:
#Read Yelp csv file
path='D:/Adarsh_Laptop/studetmater/python/pycon-2016-tutorial/data/yelp.csv'
data = pd.read_csv(path)

In [410]:
data = data[(data.stars == 5) | (data.stars == 1)] 


In [411]:
#Retreive only text and stars columns
X_data = data['text']
y_data = data['stars']

In [412]:
#checking for missing data and found none
X_data.isnull().sum()
y_data.isnull().sum()

0

In [413]:
X_data.shape

(4086L,)

In [414]:
y_data.shape

(4086L,)

In [415]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X_data,y_data,random_state=1)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(3064L,)
(1022L,)
(3064L,)
(1022L,)


In [416]:
#CountVectorizer shall be used to work on text data
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<3064x16825 sparse matrix of type '<type 'numpy.int64'>'
	with 237720 stored elements in Compressed Sparse Row format>

In [417]:
#Vectorize Test data
X_test_dtm = vect.transform(X_test)

In [418]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train=lb.fit_transform(y_train)
y_test=lb.fit_transform(y_test)

In [419]:
# Applying Naive Bayes classifier 


In [420]:
#MultinomialNB for prediction
from sklearn.naive_bayes import MultinomialNB


In [421]:
#parameter  selection and tunning
vect_cv = CountVectorizer()
cv_X_data = vect_cv.fit_transform(X_data)
cv_y_data = lb.fit_transform(y_data)
from sklearn.cross_validation import cross_val_score
alpha = np.arange(0,1,0.1)
acc_score=[]
for alpha in alpha:
    nb = MultinomialNB(alpha=alpha)
    cv_accuracy = cross_val_score(nb,cv_X_data,cv_y_data,cv=10,scoring = 'accuracy').mean()
    acc_score.append(cv_accuracy)
    

In [422]:
print max(acc_score)

0.926105581903


In [423]:
 zip(np.arange(0,1,0.1),acc_score)

[(0.0, 0.41409272719498513),
 (0.10000000000000001, 0.92194130050451084),
 (0.20000000000000001, 0.92120660270379806),
 (0.30000000000000004, 0.92316859437526788),
 (0.40000000000000002, 0.92439108826279826),
 (0.5, 0.92610558190343895),
 (0.60000000000000009, 0.92414478581061366),
 (0.70000000000000007, 0.92487648435800318),
 (0.80000000000000004, 0.92438328313669893),
 (0.90000000000000002, 0.92584907727831656)]

In [424]:
#from cross_val_score , alpha=0.5 will give the best accuracy 
nb = MultinomialNB(alpha=0.5)
nb.fit(X_train_dtm,y_train)
nb_predict = nb.predict(X_test_dtm)

In [479]:
f1_Score_NB=metrics.f1_score(y_test,nb_predict)
print f1_Score_NB

0.954707985697


In [425]:
from sklearn import metrics


In [426]:
conf_matrix= metrics.confusion_matrix(y_test,nb_predict)

In [427]:
pd.crosstab(y_test, nb_predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,145,39,184
1,37,801,838
All,182,840,1022


In [428]:
y_test[:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [429]:
lb.inverse_transform(y_test)[:5]

array([5, 5, 5, 5, 5], dtype=int64)

In [455]:
TN=conf_matrix[0,0]
FP=conf_matrix[0,1]
FN=conf_matrix[1,0]
TP=conf_matrix[1,1]

In [480]:
#Classification Accuracy
Classification_Accuracy_NB=metrics.accuracy_score(y_test,nb_predict)
Classification_Accuracy_NB

0.92563600782778865

In [481]:
#Classification Error
Classification_Error_NB=1 - metrics.accuracy_score(y_test,nb_predict)
Classification_Error_NB

0.074363992172211346

In [482]:
#Sensitivity : is the classifier to detecting positive instances , also True Positive Rate" or "Recall"
recall_NB= metrics.recall_score(y_test,nb_predict)
recall_NB

0.95584725536992843

In [483]:
#Specificity: When the actual value is negative, how often is the prediction correct? 
#How "specific" (or "selective") is the classifier in predicting positive instances?

Specificity_NB=(TN / float(TN + FP))
Specificity_NB

0.76086956521739135

In [461]:
#calculate predicted probability 
##I want probability of getting rating 5 so 
nb_predict_prob=nb.predict_proba(X_test_dtm)[:,1]

In [462]:
#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print(FP/ float(FP+TN))


0.211956521739


In [484]:
#Precision: When a positive value is predicted, how often is the prediction correct?
#How "precise" is the classifier when predicting positive instances?
print (TP/float(TP+FP))
Precision_NB= metrics.precision_score(y_test,nb_predict)

0.948235294118


In [472]:
#print rating 1 reviews which are falsely callisified as positive i.e rating 5 ,diplay False Postive review comments
print len(X_test[(nb_predict == 1) & (y_test == 0)])

#X_test[nb_predict > y_test]

39


In [335]:
#False postive review
X_test[1781]

"If you like the stuck up Scottsdale vibe this is a good place for you. The food isn't impressive. Nice outdoor seating."

In [471]:
#print rating 5 reviews which are falsely callisified as negative i.e rating 1,diplay True Negative review comments
print len(X_test[(nb_predict == 0) & (y_test == 1)])
#X_test[ nb_predict < y_test]

37


In [337]:
# True Negative , review which is falsely callasified as Rating 1
X_test[1438]

"I really enjoyed my mani/pedi with Trisha!  She took her time and taught me about manicuring my nails from home which I loved.\n\nI went in for the Groupon basic mani/pedi deal and left feeling like I got a lot more! It was the most pleasant experience I've had at a nail salon. The conversation was easy and Trish is a great person. I will be coming back to Trish for nails again! \n\nThe entire staff was friendly and professional. Great Salon!!"

In [343]:
#First 5 probabilty of getting 5 rating
nb_predict_prob[:5]

array([ 1.        ,  0.99693641,  1.        ,  0.99624713,  1.        ])

In [473]:
nb_auc= metrics.roc_auc_score(y_test,nb_predict_prob)
nb_auc

0.94623261907232536

In [491]:
val= [f1_Score_NB,Classification_Accuracy_NB,Classification_Error_NB,recall_NB,Specificity_NB,Precision_NB,nb_auc]
name=['f1_score','Classification_Accuracy','Classification_Error','Recall','Specificity','Precision','AUC']

NB_DataFrame = pd.DataFrame(zip(name,val),columns=['NB_Prop','Value'])
NB_DataFrame

,NB_Prop,Value
0,f1_score,0.954708
1,Classification_Accuracy,0.925636
2,Classification_Error,0.074364
3,Recall,0.955847
4,Specificity,0.760870
5,Precision,0.953571
6,AUC,0.946233


In [ ]:
#Solving this problem through Logistice regression
# We will try out all classifier and compare them all how fare to each other.

In [350]:
from sklearn.linear_model import LogisticRegression

In [354]:
#Parameter selection and tunning

from sklearn.cross_validation import cross_val_score
alpha = np.arange(1,5,0.1)
acc_score=[]
for alpha in alpha:
    lr = LogisticRegression(C=alpha)
    cv_accuracy = cross_val_score(lr,cv_X_data,cv_y_data,cv=10,scoring = 'accuracy').mean()
    acc_score.append(cv_accuracy)

In [355]:
print max(acc_score)

0.938823116849


In [477]:
# zip(np.arange(1,5,0.1),acc_score)

In [368]:
lr = LogisticRegression(C=1.1)
lr.fit(X_train_dtm,y_train)

LogisticRegression(C=1.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [369]:
lr_pred = lr.predict(X_test_dtm)

In [445]:
metrics.f1_score(y_test,lr_pred)

0.95497630331753547

In [377]:
#Confusion Matrix
lr_conf_matrix = metrics.confusion_matrix(y_test,lr_pred) 
#print the confusion matrix for better understanding with row and columns
pd.crosstab(y_test,lr_pred,rownames=['True'],colnames=['Predicted'],margins=True)

Predicted,0,1,All
True,,,
0,140,44,184
1,32,806,838
All,172,850,1022


In [370]:
#Classification Accuracy
print metrics.accuracy_score(y_test,lr_pred)

0.925636007828


In [381]:
#Classification Error
print 1 - metrics.accuracy_score(y_test,lr_pred)

0.0743639921722


In [383]:
#Sensitivity : is the classifier to detecting positive instances , also True Positive Rate" or "Recall"
print metrics.recall_score(y_test,lr_pred)

0.961813842482


In [465]:
TN=lr_conf_matrix[0,0]
FP=lr_conf_matrix[0,1]
FN=lr_conf_matrix[1,0]
TP=lr_conf_matrix[1,1]

In [476]:
#Specificity: When the actual value is negative, how often is the prediction correct? 
#How "specific" (or "selective") is the classifier in predicting positive instances?

Specificity_lr=(TN / float(TN + FP))
Specificity_lr

0.76086956521739135

In [467]:
#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print(FP/ float(FP+TN))

0.239130434783


In [475]:
#Precision: When a positive value is predicted, how often is the prediction correct?
#How "precise" is the classifier when predicting positive instances?
print (TP/float(TP+FP))
precision_lr=metrics.precision_score(y_test, lr_pred)
precision_lr

0.948235294118


0.94823529411764707

In [451]:
#calculate predicted probability 
##I want probability of getting rating 5 so 
lr_predict_prob=lr.predict_proba(X_test_dtm)[:,1]

In [452]:
#False positive , review which are falsely predicted as rating 5
X_test[lr_pred > y_test].count()

44

In [453]:
#True Negative , review which are falsely predicted as rating 1
X_test[lr_pred < y_test].count()

32

In [403]:
#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print(FP / float(TN + FP))

0.813953488372


In [470]:
#Calculate AUC for Logistice regression ,
auc_logistic = metrics.roc_auc_score(y_test,lr_predict_prob)
auc_logistic

0.96883755318045039

In [ ]:
#Let List out all the accuracy and validating facts from both the algorithms
#First we will go with Naive Bayes 

In [ ]:
#SVM classifier with kernel

In [495]:
from sklearn import svm
svc = svm.SVC()
svc

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [496]:
svc.fit(X_train_dtm,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [497]:
svc_predict=svc.predict(X_test_dtm)

In [498]:
#accuracy
svc_acuracy=metrics.accuracy_score(y_test,svc_predict)
svc_acuracy

0.81996086105675148